<a href="https://colab.research.google.com/github/LaoM10617/CrossLingual_InformationRetrieval2526/blob/main/validating_finewiki_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 0) Mount Drive (optional) & set cache dir
USE_DRIVE = True  #@param {type:"boolean"}

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.environ["HF_DATASETS_CACHE"] = "/content/drive/MyDrive/hf_cache"
    BASE_DIR = "/content/drive/MyDrive/mini_kept_rag"
else:
    BASE_DIR = "/content/mini_kept_rag"

import os, pathlib
pathlib.Path(BASE_DIR).mkdir(parents=True, exist_ok=True)
print("BASE_DIR =", BASE_DIR)
print("HF_DATASETS_CACHE =", os.environ.get("HF_DATASETS_CACHE"))

Mounted at /content/drive
BASE_DIR = /content/drive/MyDrive/mini_kept_rag
HF_DATASETS_CACHE = /content/drive/MyDrive/hf_cache


In [2]:
#@title 1) Install deps
!pip install datasets>=2.19.0
!pip install pandas>=2.0.0
!pip install numpy>=1.24.0
!pip install sentence-transformers>=2.7.0
!pip install tqdm>=4.66.0
!pip install faiss-cpu
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.8 MB/s eta 0:00:00


In [3]:
#@title 2) Embedding Filter
import re

LANGS = ["en","de","zh","ar","fr"]
PER_LANG_MAX = 2000
MIN_TOK, MAX_TOK = 20, 300

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch

DEVICE = "cuda"
EMB_MODEL_NAME = "intfloat/multilingual-e5-base"
emb_model = SentenceTransformer(EMB_MODEL_NAME, device=DEVICE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
TOPIC_SEEDS_POS = {
    "en": [
        "This text is about culture and traditions.",
        "The passage describes family celebrations and holidays.",
        "It talks about traditional food and festival customs.",
        "The text explains religious festivals and cultural rituals.",
        "This document focuses on weddings, New Year and other cultural events.",
    ],
    "de": [
        "Dieser Text handelt von Kultur und Traditionen.",
        "Der Abschnitt beschreibt Familienfeiern und Feiertage.",
        "Es geht um traditionelles Essen und Festbräuche.",
        "Der Text erklärt religiöse Feste und kulturelle Rituale.",
        "Dieses Dokument konzentriert sich auf Hochzeiten, Weihnachten und andere kulturelle Ereignisse.",
    ],
    "zh": [
        "这段文字讨论文化和传统。",
        "文章描述家庭聚会和节日庆祝。",
        "内容涉及传统食物和节日习俗。",
        "这段话介绍宗教节日和文化仪式。",
        "文本重点是婚礼、春节等文化活动。",
    ],
    "ar": [
        "يتناول هذا النص الثقافة والتقاليد.",
        "تصف الفقرة الاحتفالات العائلية والأعياد.",
        "يتحدث عن الطعام التقليدي وعادات المهرجانات.",
        "يشرح النص الأعياد الدينية والطقوس الثقافية.",
        "يركز هذا المستند على حفلات الزواج ورمضان وغيرها من المناسبات الثقافية.",
    ],
    "fr": [
        "Ce texte parle de culture et de traditions.",
        "Le passage décrit des fêtes familiales et des jours fériés.",
        "Il s'agit de cuisine traditionnelle et de coutumes de festival.",
        "Le texte explique des fêtes religieuses et des rituels culturels.",
        "Ce document se concentre sur les mariages, Noël et d'autres événements culturels.",
    ],
}

TOPIC_SEEDS_NEG = {
    "en": [
        "This text is about physics and mathematics.",
        "The passage describes financial markets and stock prices.",
        "It explains computer hardware and software engineering.",
        "The document focuses on chemistry experiments.",
        "This article is about astronomy and planets.",
    ],
    "de": [
        "Dieser Text handelt von Physik und Mathematik.",
        "Der Abschnitt beschreibt Finanzmärkte und Aktienkurse.",
        "Es geht um Computerhardware und Softwareentwicklung.",
        "Das Dokument fokussiert sich auf Chemieexperimente.",
        "Der Artikel handelt von Astronomie und Planeten.",
    ],
    "zh": [
        "这段文字讨论物理和数学。",
        "文章描述金融市场和股票价格。",
        "内容涉及计算机硬件和软件工程。",
        "文本重点是化学实验。",
        "这篇文章介绍天文学和行星。",
    ],
    "ar": [
        "يتناول هذا النص الفيزياء والرياضيات.",
        "تصف الفقرة الأسواق المالية وأسعار الأسهم.",
        "يشرح النص عتاد الحاسوب وهندسة البرمجيات.",
        "يركز المستند على تجارب الكيمياء.",
        "تتحدث هذه المقالة عن علم الفلك والكواكب.",
    ],
    "fr": [
        "Ce texte parle de physique et de mathématiques.",
        "Le passage décrit les marchés financiers et les cours des actions.",
        "Il explique le matériel informatique et l'ingénierie logicielle.",
        "Le document se concentre sur des expériences de chimie.",
        "Cet article traite d'astronomie et de planètes.",
    ],
}

# 确保每种语言都有 seed
for lang in LANGS:
    assert lang in TOPIC_SEEDS_POS and lang in TOPIC_SEEDS_NEG, f"Missing seeds for {lang}"

In [22]:
TOPIC_EMB_POS = {}
TOPIC_EMB_NEG = {}

def build_topic_embeddings():
    print("Building topic embeddings ...")
    for lang in LANGS:
        pos = TOPIC_SEEDS_POS[lang]
        neg = TOPIC_SEEDS_NEG[lang]

        pos_emb = emb_model.encode(
            pos, convert_to_numpy=True, normalize_embeddings=True
        )
        neg_emb = emb_model.encode(
            neg, convert_to_numpy=True, normalize_embeddings=True
        )

        TOPIC_EMB_POS[lang] = pos_emb.mean(axis=0)
        TOPIC_EMB_NEG[lang] = neg_emb.mean(axis=0)

        # normalization
        TOPIC_EMB_POS[lang] /= np.linalg.norm(TOPIC_EMB_POS[lang]) + 1e-9
        TOPIC_EMB_NEG[lang] /= np.linalg.norm(TOPIC_EMB_NEG[lang]) + 1e-9

        print(f"{lang}: pos/neg topic vectors built.")

build_topic_embeddings()

# softmax topic score
def topic_score(text: str, lang: str) -> float:
    """
    p_topic \in [0,1]
    cos(pos), cos(neg) -> softmax -> p_topic
    """
    text = (text or "").strip()
    if not text:
        return 0.0
    inp = f"passage: {text[:1000]}"
    emb = emb_model.encode(
        [inp], convert_to_numpy=True, normalize_embeddings=True
    )[0]

    sim_pos = float(np.dot(emb, TOPIC_EMB_POS[lang]))
    sim_neg = float(np.dot(emb, TOPIC_EMB_NEG[lang]))

    # softmax over [sim_pos, sim_neg]
    logits = np.array([sim_pos, sim_neg], dtype=np.float32)
    logits = logits - logits.max()
    exp = np.exp(logits)
    probs = exp / exp.sum()

    p_topic = float(probs[0])
    return p_topic


def tok_count(text: str, lang: str) -> int:
    text = text or ""
    if lang in ["zh", "ja", "ko"]:
        return len(text)
    else:
        return len(text.split())


SCORE_SAMPLES = {lang: [] for lang in LANGS}
MAX_SAMPLES_PER_LANG = 100

# 你可以先设一个保守阈值，之后根据 histogram 再调
MIN_TOPIC_SCORE = 0.49
print("MIN_TOPIC_SCORE =", MIN_TOPIC_SCORE)

Building topic embeddings ...
en: pos/neg topic vectors built.
de: pos/neg topic vectors built.
zh: pos/neg topic vectors built.
ar: pos/neg topic vectors built.
fr: pos/neg topic vectors built.
MIN_TOPIC_SCORE = 0.49


<>:31: SyntaxWarning: invalid escape sequence '\i'
<>:31: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipython-input-3000518098.py:31: SyntaxWarning: invalid escape sequence '\i'
  返回当前文本与【文化主题】的相似度概率 p_topic \in [0,1]


In [23]:
def keep_example(ex, lang: str) -> bool:
    text = (ex.get("title") or "") + " " + (ex.get("text") or "")

    p = topic_score(text, lang)

    samples = SCORE_SAMPLES[lang]
    if len(samples) < MAX_SAMPLES_PER_LANG:
        samples.append(p)

    n_tok = tok_count(text, lang)
    if not (MIN_TOK <= n_tok <= MAX_TOK):
        return False

    return p >= MIN_TOPIC_SCORE

In [24]:
#@title 3) Stream finewiki & write per-language JSONL
import os, json
from tqdm import tqdm
from datasets import load_dataset
from unidecode import unidecode

mini_dir = f"{BASE_DIR}/mini_finewiki"
os.makedirs(mini_dir, exist_ok=True)

for lang in LANGS:
    print(f"[{lang}] streaming…")
    ds = load_dataset("HuggingFaceFW/finewiki", lang, split="train", streaming=True)
    out_path = f"{mini_dir}/{lang}.jsonl"
    kept, buf, BATCH = 0, [], 512
    with open(out_path, "w", encoding="utf-8") as f:
        for ex in tqdm(ds, total=None):
            if keep_example(ex,lang):
                row = {
                    "in_language": lang,
                    "title": (ex.get("title") or "").strip(),
                    "text": (ex.get("text") or "").strip(),
                    "wikidata_id": (ex.get("wikidata_id") or "").strip()
                }
                buf.append(json.dumps(row, ensure_ascii=False))
                if len(buf) >= BATCH:
                    f.write("\n".join(buf) + "\n")
                    buf.clear()
                kept += 1
                if kept >= PER_LANG_MAX:
                    break
        if buf:
            f.write("\n".join(buf) + "\n")
    print(f"[{lang}] kept = {kept}, saved → {out_path}")

[en] streaming…


0it [00:00, ?it/s]'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 17154cb4-81ee-4aa0-a503-344161fde37d)')' thrown while requesting GET https://huggingface.co/datasets/HuggingFaceFW/finewiki/resolve/8bd13e72e6a002407649b3e898535f42ceb1aeb9/data/enwiki/000_00000.parquet
Retrying in 1s [Retry 1/5].
4397it [00:57, 76.13it/s]


[en] kept = 2000, saved → /content/drive/MyDrive/mini_kept_rag/mini_finewiki/en.jsonl
[de] streaming…


3521it [00:46, 76.39it/s]


[de] kept = 2000, saved → /content/drive/MyDrive/mini_kept_rag/mini_finewiki/de.jsonl
[zh] streaming…


...

[zh] kept = 2000, saved → /content/drive/MyDrive/mini_kept_rag/mini_finewiki/zh.jsonl
[ar] streaming…


2336it [00:31, 75.18it/s]


[ar] kept = 2000, saved → /content/drive/MyDrive/mini_kept_rag/mini_finewiki/ar.jsonl
[fr] streaming…


17443it [03:37, 80.21it/s]

[fr] kept = 2000, saved → /content/drive/MyDrive/mini_kept_rag/mini_finewiki/fr.jsonl


In [25]:
# check score distribution to adjust threshold
for lang in LANGS:
    scores = np.array(SCORE_SAMPLES[lang], dtype=np.float32)
    if len(scores) == 0:
        print(f"[{lang}] no scores recorded.")
        continue
    print(f"\n[{lang}] score stats (p_topic):")
    print("  n_samples:", len(scores))
    print("  min / max:", float(scores.min()), float(scores.max()))
    for q in [10, 25, 50, 75, 90]:
        print(f"  p{q:02d}:", float(np.percentile(scores, q)))


[en] score stats (p_topic):
  n_samples: 100
  min / max: 0.48670876026153564 0.5021951794624329
  p10: 0.48971280455589294
  p25: 0.49348706007003784
  p50: 0.4954836368560791
  p75: 0.49760255217552185
  p90: 0.49939972162246704

[de] score stats (p_topic):
  n_samples: 100
  min / max: 0.4887477457523346 0.5028334856033325
  p10: 0.49230992794036865
  p25: 0.49430227279663086
  p50: 0.4964544177055359
  p75: 0.4987952411174774
  p90: 0.5005126595497131

[zh] score stats (p_topic):
  n_samples: 100
  min / max: 0.4898189902305603 0.500827431678772
  p10: 0.49403834342956543
  p25: 0.4964147210121155
  p50: 0.49722129106521606
  p75: 0.49790868163108826
  p90: 0.4987408518791199

[ar] score stats (p_topic):
  n_samples: 100
  min / max: 0.4907696545124054 0.49927541613578796
  p10: 0.4919094741344452
  p25: 0.49229884147644043
  p50: 0.49265486001968384
  p75: 0.4930615723133087
  p90: 0.4935818314552307

[fr] score stats (p_topic):
  n_samples: 100
  min / max: 0.48441803455352783 0

# Preview for each language

In [26]:
def preview(path, k=5):
    print(f"=== Preview: {path} ===")
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= k:
                break
            ex = json.loads(line)
            print(f"\n[{i}] title={ex['title'][:50]}")
            print(f"text[:120]={ex['text'][:120]}")
    print("-"*60)

langs = ["en", "de", "zh", "ar", "fr"]
base = "/content/drive/MyDrive/mini_kept_rag/mini_finewiki"

for lang in langs:
    path = f"{base}/{lang}.jsonl"
    preview(path)

=== Preview: /content/drive/MyDrive/mini_kept_rag/mini_finewiki/en.jsonl ===

[0] title=135th Indiana Infantry Regiment
text[:120]=# 135th Indiana Infantry Regiment
The 135th Indiana Infantry Regiment served in the Union Army between May 23 and Septem

[1] title=135th Illinois Infantry Regiment
text[:120]=# 135th Illinois Infantry Regiment
The 135th Illinois Infantry Regiment was an infantry regiment from Illinois that serv

[2] title=1420 in Ireland
text[:120]=# 1420 in Ireland
Events from the year 1420 in Ireland.

## Incumbent
- Lord: Henry V


## Events
- Luttrellstown Castle

[3] title=1636 in France
text[:120]=# 1636 in France
Events from the year 1636 in France

## Incumbents
- Monarch – Louis XIII[1]


## Events
- 20 March – T

[4] title=1656 in Denmark
text[:120]=# 1656 in Denmark
Events from the year 1656 in Denmark.

## Incumbents
- Monarch – Frederick III[1]
- Steward of the Rea
------------------------------------------------------------
=== Preview: /content/drive/MyDrive